In [1]:
# we need a way to specify unique model name, or save model outputs based on indices.

import numpy as np
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from model_data_definitions import *
from concurrent.futures import ThreadPoolExecutor

# expensive and slow operation done once.
filtered_datasets, datasets = fetch_return_filtered_pmlb_data_sets() 
print(filtered_datasets)


    dataset  n_instances  n_features  n_binary_features  \
2  1029_LEV         1000           4                  0   
3  1030_ERA         1000           4                  0   

   n_categorical_features  n_continuous_features endpoint_type  n_classes  \
2                       0                      4    continuous        5.0   
3                       0                      4    continuous        9.0   

   imbalance        task  
2   0.111245  regression  
3   0.031251  regression  


In [2]:
models = initialize_all_models(input_dimension=2, seed_val=42)

compile_models(models)

for model, name in models:
    # Access the model and its unique name
    print(name)
    #print(model.summary())

Linear Model
Wide ReLU ANN
Deep ReLU ANN
One Parameter
Spline ANN (z=1)
Lookup Table (z=1)
ABEL-Spline (z=1)
Spline ANN (z=2)
Lookup Table (z=2)
ABEL-Spline (z=2)
Spline ANN (z=4)
Lookup Table (z=4)
ABEL-Spline (z=4)
Spline ANN (z=8)
Lookup Table (z=8)
ABEL-Spline (z=8)
Spline ANN (z=10)
Lookup Table (z=10)
ABEL-Spline (z=10)


In [3]:
initialize_all_models

<function model_data_definitions.initialize_all_models(input_dimension: int, seed_val: int, output_dim: int = 1, hidden_units_wide: int = 1000, hidden_units_deep: int = 16, hidden_layers: int = 8, num_exps: int = 6) -> list>

In [34]:
[(train_index, test_index) for train_index,test_index in kf.split(X)]

[(array([  0,   1,   2,   4,   5,   6,   7,   8,  10,  11,  14,  16,  17,
          18,  19,  20,  21,  23,  25,  26,  27,  28,  29,  33,  34,  35,
          36,  38,  42,  43,  44,  45,  46,  49,  50,  51,  53,  54,  56,
          57,  58,  59,  62,  65,  66,  67,  68,  69,  71,  72,  76,  78,
          79,  80,  81,  82,  84,  85,  93,  95,  96,  97,  98, 100, 102,
         103, 105, 106, 107, 110, 111, 112, 113, 117, 118, 119, 120, 121,
         122, 123, 124, 126, 127, 129, 133, 135, 136, 138, 140, 142, 143,
         144, 145, 147, 148, 149, 151, 152, 154, 155, 162, 163, 164, 165,
         166, 168, 171, 172, 173, 174, 175, 177, 181, 182, 183, 184, 185,
         186, 188, 189, 192, 197, 198, 199, 200, 202, 203, 204, 205, 208,
         209, 210, 212, 213, 214, 215, 218, 219, 220, 221, 223, 224, 225,
         227, 230, 232, 233, 234, 235, 236, 238, 242, 244, 245, 246, 247,
         248, 250, 252, 253, 254, 256, 257, 258, 259, 261, 262, 265, 266,
         267, 268, 269, 272, 273, 274,

In [28]:
kf = KFold(n_splits=3,shuffle=True)


In [32]:
for train_index, test_index in kf.split(X):
    print(test_index)


[  7  15  16  21  23  27  31  34  36  38  40  43  44  45  46  47  48  52
  53  55  56  63  68  70  71  81  90  92  97 100 108 113 114 116 117 118
 119 121 122 123 125 126 130 131 133 134 135 138 140 142 150 155 162 163
 164 166 167 170 171 178 180 182 183 186 190 193 198 199 200 203 204 208
 209 210 211 215 219 220 224 225 228 230 233 243 245 246 247 254 258 262
 263 271 273 274 275 276 278 281 286 291 295 296 300 306 307 317 318 322
 327 337 338 339 350 351 353 354 362 364 369 370 371 373 379 381 384 386
 388 392 393 397 398 399 401 405 407 408 410 412 414 415 420 421 423 425
 433 438 442 444 448 457 458 460 463 464 465 466 468 470 472 474 475 477
 480 484 486 495 496 498 499 501 504 508 509 510 518 520 527 533 534 535
 536 537 539 541 543 544 545 549 553 554 557 558 576 585 586 587 588 589
 595 597 602 608 609 613 614 615 617 618 619 625 626 630 633 634 635 636
 638 640 643 645 646 647 648 650 652 654 657 666 669 672 673 677 679 683
 694 695 697 699 701 702 714 715 718 719 728 733 73

In [35]:
input_dimension

4

In [7]:
import json

all_results = {}
epoch_number = 1

for dataset, row in zip(datasets, filtered_datasets.iterrows()):
    dataset_name = row[1]['dataset']
    print(f"Evaluating dataset: {dataset_name}")

    input_dimension = row[1]['n_features']
    #models = initialize_all_models(input_dimension)
    #compile_models(models)
    
    data = dataset

    X, y = data.drop('target', axis=1).values, data['target'].values

    results = []
    kf = KFold(n_splits=2,shuffle=True)
    fold = 0

    for train_index, test_index in kf.split(X):
        fold += 1
        print(f"Fold #{fold}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and compile models with unique seed value for each fold
        models = initialize_all_models(X_train.shape[1], seed_val=fold)
        compile_models(models)

        for model, name in models:
            history = model.fit(X_train, y_train,
                                epochs=epoch_number,
                                verbose=0,
                                validation_data=(X_test, y_test))

            # Get loss and accuracy on test data
            #loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
            loss = model.evaluate(X_test, y_test, verbose=0)

            # Make predictions to calculate R squared value and test error
            predictions = model.predict(X_test)
            r_squared_value = r2_score(y_test,predictions)
            test_error = mean_squared_error(y_test,predictions)

            results.append({
                'model': name,
                'fold': fold,
                'history': history,
                'loss': loss,
                'r_squared_value': r_squared_value,
                'test_error': test_error})


Evaluating dataset: 1029_LEV
Fold #1
16/16 [==============================] - 0s 1000us/step
Fold #2
16/16 [==============================] - 0s 1ms/step
Evaluating dataset: 1030_ERA
Fold #1
16/16 [==============================] - 0s 1ms/step
Fold #2
16/16 [==============================] - 0s 1000us/step


In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import tensorflow as tf

def cross_validation(data, epoch_number=2):
    X, y = data.drop('target', axis=1).values, data['target'].values
    
    results = []
    kf = KFold(n_splits=2)
    fold = 0

    for train_index, test_index in kf.split(X):
        fold += 1
        print(f"Fold #{fold}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and compile models with unique seed value for each fold
        models = initialize_all_models(X_train.shape[1], seed_val=fold)
        compile_models(models)

        for model, name in models:
            history = model.fit(X_train, y_train,
                                epochs=epoch_number,
                                verbose=0,
                                validation_data=(X_test, y_test))
            
            # Get loss and accuracy on test data
            #loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
            loss = model.evaluate(X_test, y_test, verbose=0)
            
            # Make predictions to calculate R squared value and test error
            predictions = model.predict(X_test)
            r_squared_value = r2_score(y_test,predictions)
            test_error = mean_squared_error(y_test,predictions)

            results.append({
                'model': name,
                'fold': fold,
                'history': history,
                'loss': loss,
                'r_squared_value': r_squared_value,
                'test_error': test_error})

    return results

In [5]:
import json

all_results = {}

for dataset, row in zip(datasets, filtered_datasets.iterrows()):
    dataset_name = row[1]['dataset']
    print(f"Evaluating dataset: {dataset_name}")

    input_dimension = row[1]['n_features']
    #models = initialize_all_models(input_dimension)
    #compile_models(models)

    results = cross_validation(dataset)
    all_results[dataset_name] = results

# save results to a JSON file
with open('results.json', 'w') as f:
    json.dump(all_results, f)

Evaluating dataset: 1029_LEV
Fold #1
16/16 [==============================] - 0s 1000us/step
Fold #2
16/16 [==============================] - 0s 1ms/step
Evaluating dataset: 1030_ERA
Fold #1
16/16 [==============================] - 0s 1ms/step
Fold #2
16/16 [==============================] - 0s 1ms/step


TypeError: Object of type History is not JSON serializable

In [ ]:
all_results = {}

for dataset, row in zip(datasets, filtered_datasets.iterrows()):
    dataset_name = row[1]['dataset']
    print(f"Evaluating dataset: {dataset_name}")

    input_dimension = row[1]['n_features']
    models = initialize_all_models(input_dimension, seed_val=42)
    compile_models(models)

    results = cross_val(models, dataset)
    all_results[dataset_name] = results

print(all_results)

In [8]:
import json

# Create a list of names for datasets locally, along with the number of epochs for training, and the model names
dataset_names = filtered_datasets['dataset'].tolist()
epochs = 10
model_names = [name for _, name in models]

# Define experiment settings
experiment_settings = {
    'dataset_names': dataset_names,
    'epochs': epochs,
    'model_names': model_names
}

# Check if the file for experimental setup exists and whether it matches the current experimental variable
try:
    with open('experimental_setup.json', 'r') as f:
        saved_experiment_settings = json.load(f)
except FileNotFoundError:
    saved_experiment_settings = None

if saved_experiment_settings != experiment_settings:
    # If there are differences or the text file does not exist, then create and save your description to a text file with the same name.
    with open('experimental_setup.json', 'w') as f:
        json.dump(experiment_settings, f)

    # If experimental settings have changed, wipe progress clean
    completed_datasets = []
else:
    # If experimental setup has not changed, load progress from previous run (if it exists)
    try:
        with open('progress.json', 'r') as f:
            completed_datasets = json.load(f)
    except FileNotFoundError:
        completed_datasets = []
        
all_results = {}

for dataset, row in zip(datasets, filtered_datasets.iterrows()):
    dataset_name = row[1]['dataset']
    
    if dataset_name in completed_datasets:  # Skip already processed datasets
        continue

    print(f"Evaluating dataset: {dataset_name}")

    input_dimension = row[1]['n_features']
    
    models = initialize_all_models(input_dimension=input_dimension,
                                   seed_val=42)

    compile_models(models)

    results = cross_val(models=models,
                        data=dataset)

    
    all_results[dataset_name] = results

    # Add dataset to the list of completed datasets and save progress
    completed_datasets.append(dataset_name)
    with open('progress.json', 'w') as f:
        json.dump(completed_datasets, f)

print(all_results)

Evaluating dataset: 1030_ERA
4/4 [==============================] - 0s 981us/step


4/4 [==============================] - 0s 1ms/step
{'1030_ERA': {'Linear Model': [3.8531771981716156, 3.819590944945812, 3.7945997408032417, 3.1281222593784332, 2.861634168624878, 2.398332995772362, 1.9819897782802582, 1.9569971477985382, 2.015181655883789, 1.8134105348587035], 'Wide ReLU ANN': [1.1086437726020812, 1.3716701412200927, 1.2909508776664733, 1.2591666722297667, 1.1933196902275085, 1.0947322845458984, 1.2583980798721313, 1.1224033093452455, 1.310504549741745, 1.2581566095352172], 'Deep ReLU ANN': [1.1132738101482391, 1.4158685731887817, 1.3811927342414856, 1.2575048041343688, 1.1670758676528932, 1.0914067101478577, 1.2408037304878234, 1.1180108380317688, 1.3448182368278503, 1.2617321062088012], 'One Parameter': [3.8217566275596617, 3.7517603611946106, 3.8117640948295595, 3.1428709745407106, 2.857427086830139, 2.4835174036026, 2.064221992492676, 2.111298151016235, 2.1524038219451906, 1.9879837465286254], 'Spline ANN (z=1)': [3.0057036113739013, 2.420246036052704, 2.165396399

In [7]:
# Create a DataFrame to store the results
result_data = []

# Iterate through all_results and compute mean and std for each model
for model_name in all_results[list(all_results.keys())[0]].keys():
    all_maes = [np.mean(all_results[dataset][model_name]) for dataset in all_results]
    avg_mae = np.mean(all_maes)
    std_mae = np.std(all_maes)
    result_data.append([model_name, avg_mae, std_mae])

# Convert the results to a pandas DataFrame
results_df = pd.DataFrame(result_data, columns=['Model', 'Average MAE', 'Std Deviation'])
print(results_df)

# Add R-squared error,

                  Model  Average MAE  Std Deviation
0          Linear Model     0.817498            0.0
1         Wide ReLU ANN     0.438172            0.0
2         Deep ReLU ANN     0.427755            0.0
3         One Parameter     1.027055            0.0
4      Spline ANN (z=1)     0.578913            0.0
5    Lookup Table (z=1)     1.026886            0.0
6     ABEL-Spline (z=1)     0.478548            0.0
7      Spline ANN (z=2)     0.574952            0.0
8    Lookup Table (z=2)     0.981407            0.0
9     ABEL-Spline (z=2)     0.471621            0.0
10     Spline ANN (z=4)     0.573082            0.0
11   Lookup Table (z=4)     1.172064            0.0
12    ABEL-Spline (z=4)     0.473223            0.0
13     Spline ANN (z=8)     0.580037            0.0
14   Lookup Table (z=8)     1.247612            0.0
15    ABEL-Spline (z=8)     0.470044            0.0
16    Spline ANN (z=10)     0.572392            0.0
17  Lookup Table (z=10)     1.243610            0.0
18   ABEL-Sp

In [9]:

def cross_val(models, data):
    X, y = data.drop('target', axis=1).values, data['target'].values

    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    results = {}

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train, X_test = preprocess_data(X_train, X_test)

        for model, model_name in models:
            #model_name = type(model).__name__
            model.fit(X_train, y_train, epochs = 10, verbose=0)
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)

            if model_name not in results:
                results[model_name] = []

            results[model_name].append(mae)
    
    return results

all_results = {}

for dataset, row in zip(datasets, filtered_datasets.iterrows()):
    dataset_name = row[1]['dataset']
    print(f"Evaluating dataset: {dataset_name}")

    input_dimension = row[1]['n_features']
    models = initialize_all_models(input_dimension, seed_val=42)
    compile_models(models)

    results = cross_val(models, dataset)
    all_results[dataset_name] = results

print(all_results)

Evaluating dataset: 1029_LEV
4/4 [==============================] - 0s 1ms/step
Evaluating dataset: 1030_ERA
4/4 [==============================] - 0s 1ms/step


4/4 [==============================] - 0s 1ms/step
{'1029_LEV': {'Linear Model': [1.768871571570635, 1.3707225424051286, 1.1655029916763306, 0.7663325721025467, 0.6250245231389999, 0.5596683585643768, 0.4607035970687866, 0.5361593437194824, 0.5138296473026276, 0.43046897649765015], 'Wide ReLU ANN': [0.5617390830814838, 0.49351791605353357, 0.47327164143323897, 0.4149248341470957, 0.4226074993610382, 0.3848393700644374, 0.37083969965577124, 0.4398356653377414, 0.4567626665905118, 0.36179728135466577], 'Deep ReLU ANN': [0.5625948396697641, 0.5162016713619232, 0.47287281915545465, 0.3998201033473015, 0.3696572086215019, 0.35344279382377863, 0.3876857726275921, 0.41459656819701196, 0.42809998489916323, 0.34474669583141804], 'Lookup Table Model': [1.709847707748413, 1.4336150300502777, 1.1501507759094238, 0.8434668397903442, 0.9108668851852417, 0.9290204620361329, 0.7777738285064697, 0.8783207058906555, 0.8939595031738281, 0.7260821342468262], 'Spline ANN (Partition 1)': [1.078141376376152,

In [10]:
# Create a DataFrame to store the results
result_data = []

# Iterate through all_results and compute mean and std for each model
for model_name in all_results[list(all_results.keys())[0]].keys():
    all_maes = [np.mean(all_results[dataset][model_name]) for dataset in all_results]
    avg_mae = np.mean(all_maes)
    std_mae = np.std(all_maes)
    result_data.append([model_name, avg_mae, std_mae])

# Convert the results to a pandas DataFrame
results_df = pd.DataFrame(result_data, columns=['Model', 'Average MAE', 'Std Deviation'])
print(results_df)

# Add R-squared error,

                                Model  Average MAE  Std Deviation
0                        Linear Model     1.790073       0.970345
1                       Wide ReLU ANN     0.835676       0.397663
2                       Deep ReLU ANN     0.838483       0.413511
3                  Lookup Table Model     1.921736       0.896426
4            Spline ANN (Partition 1)     1.166229       0.587540
5    Lookup Table Model (Partition 1)     1.922888       0.895788
6           ABEL Spline (Partition 1)     0.873222       0.394664
7            Spline ANN (Partition 2)     1.158109       0.584776
8    Lookup Table Model (Partition 2)     1.985156       1.004764
9           ABEL Spline (Partition 2)     0.864738       0.392545
10           Spline ANN (Partition 4)     1.164674       0.591706
11   Lookup Table Model (Partition 4)     2.157457       0.984095
12          ABEL Spline (Partition 4)     0.859736       0.387231
13           Spline ANN (Partition 8)     1.181812       0.600596
14   Looku

In [ ]:
all_results['1029_LEV'][]

In [2]:
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import pmlb
from pmlb import fetch_data
from model_definitions import *

def compile_models(models, optimizer='adam', loss='mean_absolute_error'):
    """Compile TensorFlow/Keras models."""
    for model in models:
        model.compile(optimizer=optimizer, loss=loss)


def preprocess_data(train_data, test_data):
    """Preprocess the data by zero-centering, scaling to unit variance, and applying a sigmoid."""
    bias = np.mean(train_data, axis=0)
    variance = np.std(train_data, axis=0)
    
    train_data = (train_data - bias) / variance
    test_data = (test_data - bias) / variance
    
    train_data = 1 / (1 + np.exp(-train_data))
    test_data = 1 / (1 + np.exp(-test_data))
    
    return train_data, test_data


def cross_val(models, dataset_name, n_splits=10):
    data = fetch_data(dataset_name)
    X, y = data.drop('target', axis=1).values, data['target'].values

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = {}

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train, X_test = preprocess_data(X_train, X_test)

        for model in models:
            model_name = type(model).__name__
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            mae = mean_absolute_error(y_test, y_pred)

            if model_name not in results:
                results[model_name] = []

            results[model_name].append(mae)
    
    return results


def initialize_all_models(input_dimension: int, 
                          seed_val: int, 
                          output_dim: int = 1,
                          hidden_units_wide: int = 1000,
                          hidden_units_deep: int = 16,
                          hidden_layers: int = 8,
                          num_exps: int = 6) -> list:
    """Initialize models with given configurations."""
    common_args = {
        'input_dim': input_dimension, 
        'output_dim': output_dim, 
        'seed': seed_val
    }

    models = [
        create_linear_model(**common_args),
        create_wide_relu_ann(hidden_units=hidden_units_wide, **common_args),
        create_deep_relu_ann(hidden_units=hidden_units_deep, hidden_layers=hidden_layers, **common_args),
        LookupTableModel(partition_num=1, default_val=-1., **common_args)
    ]

    for partition_num in [1,2,4,8,10]:
        models.append(SplineANN(partition_num=partition_num, **common_args))
        models.append(LookupTableModel(partition_num=partition_num, default_val=-1., **common_args))
        models.append(ANNEXSpline(partition_num=partition_num, num_exps=num_exps, **common_args))

    return models


# Path to the TSV file
metadata_path = os.path.join(os.path.dirname(pmlb.__file__), 'all_summary_stats.tsv')
metadata = pd.read_csv(metadata_path, sep='\t')

# Filter datasets based on number of records (rows) and other criteria
filtered_datasets = metadata[
    (metadata['n_features'] < 5) &
    (metadata['n_binary_features'] == 0) &
    (metadata['n_categorical_features'] == 0) &
    (metadata['n_continuous_features'] == metadata['n_features'] ) &
    (metadata['n_instances'] >= 500) &  # Assuming 'n_instances' is the column for the number of records
    (metadata['n_instances'] <= 1000) &
    (metadata['endpoint_type'] == 'continuous') &
    (metadata['task'] == 'regression')
]

all_results = {}

for _, row in filtered_datasets.iterrows():
    dataset_name = row['dataset']
    print(f"Evaluating dataset: {dataset_name}")
    
    input_dimension = row['n_features']
    models = initialize_all_models(input_dimension, seed_val=42)
    compile_models(models)
    
    results = cross_val(models, dataset_name)
    all_results[dataset_name] = results

print(all_results)

# Please wrap the filtering and data fetching in a separate function. I want the **data set** itself to be given as an input to the 
# cross_val function. Here is an example:

# Function to fetch a dataset by name
def fetch_dataset(row):
    dataset_name = row[1][0]  # Adjust this based on the actual structure
    #print(dataset_name)
    data = fetch_data(dataset_name)
    # Further processing if needed
    return data

# Fetch the filtered datasets in parallel
with ThreadPoolExecutor() as executor:
    datasets = list(executor.map(fetch_dataset, filtered_datasets.iterrows()))

# 'datasets' now contains the datasets that meet your criteria I want to contain/encapsulate the PMLB dependent code
# in one function to make my life easier if I wat to use other data se




Evaluating dataset: 1029_LEV
29/29 [==============================] - 0s 647us/step - loss: 1.6704


4/4 [==============================] - 0s 1ms/step
Evaluating dataset: 1030_ERA
4/4 [==============================] - 0s 1ms/step


4/4 [==============================] - 0s 1ms/step
{'1029_LEV': {'Sequential': [2.0221621349453924, 0.7933430811762809, 1.6941675129532814, 1.874968957975507, 0.5584732055664062, 0.7524310970306396, 1.9036995647475123, 0.5179093956947327, 0.589298689365387, 1.618078442439437, 0.4877166260778904, 0.511274830698967, 1.7129278630018234, 0.44329070568084716, 0.49818726181983947, 1.7118319395929575, 0.4322559240460396, 0.45962959721684454, 1.4660055024921894, 0.4231223513931036, 0.4359669440239668, 1.6609746345877647, 0.4848525023832917, 0.5162886396795511, 1.8431587083637715, 0.49203173197805883, 0.5089119470492005, 1.490471440255642, 0.40738728769123556, 0.44686184383928773], 'LookupTableModel': [1.9142543955240399, 1.9142684667930008, 1.905387876444729, 1.9080076811497566, 1.9122485671692993, 1.9110672793351113, 1.8899396319687367, 1.8899350018054246, 1.8847575262491592, 1.8885278181289322, 1.9034409697019146, 1.8985536113928538, 1.8081036318838597, 1.8083110586553812, 1.8014922329195542

In [ ]:
# Function to fetch a dataset by name
def fetch_dataset(row):
    dataset_name = row[1][0]  # Adjust this based on the actual structure
    #print(dataset_name)
    data = fetch_data(dataset_name)
    # Further processing if needed
    return data

# Fetch the filtered datasets in parallel
with ThreadPoolExecutor() as executor:
    datasets = list(executor.map(fetch_dataset, filtered_datasets.iterrows()))

# 'datasets' now contains the datasets that meet your criteria
